In [1]:
# Imports
import requests
import json
import os 

In [2]:
#Config 
'''
  Chirpstack API connection
'''
username = "admin"
password = "lora@0309"
url = "http://localhost:8080/api"

'''
   Expirementation config
'''
EXP_FOLDER = 'exp/'


In [3]:
def connect():
    
    ''' 
        Chirpstack API Connection 
        returns authorization token
    '''

    headers = {
        "Content-Type": "application/json",
        "Accept": "application/json",
    }

    data = '{ "password": "' + password + '", \n   "email": "' + username + '" \n }'

    response = requests.post(f"{url}/internal/login", headers=headers, data=data)
    r = json.loads(response.content.decode())
    try:
        return r["jwt"]
        # print(self.token)
    except:
        return None

    
def startGatewayFrameHandler(gatewayid):
    
    token = connect()
    
    if token :
        
        headers = {
            "Accept": "application/json",
            "Grpc-Metadata-Authorization": f"Bearer {token}",
        }


        with requests.get(f'{url}/gateways/{gatewayid}/frames', headers=headers, stream=True) as f:
            print(f)
            for l in f.iter_lines():
                if l:
                    decoded = l.decode()
                    trimmed = (
                        decoded.replace('\\"', '"')
                        .replace('"{', "{")
                        .replace('}"', "}")
                    )
                    data = json.loads(trimmed)
                    print(data)
    else:
        print('Authorization Required')
    

def startDeviceFrameHandler(devEUI,exp):
    
    token = connect()
    
    if token :
        
        headers = {
            "Accept": "application/json",
            "Grpc-Metadata-Authorization": f"Bearer {token}",
        }
        if not os.path.exists(f'{EXP_FOLDER}{exp}'):
            os.mkdir(f'{EXP_FOLDER}{exp}')
        exp_data = []
        #downlinks =[]
        with requests.get(f'{url}/devices/{devEUI}/frames', headers=headers, stream=True) as f:
            #print(f)
            for l in f.iter_lines():
                if l:
                    decoded = l.decode()
                    trimmed = (
                        decoded.replace('\\"', '"')
                        .replace('"{', "{")
                        .replace('}"', "}")
                    )
                    data = json.loads(trimmed)
                    
                    data_path = f'{EXP_FOLDER}{exp}/data_{devEUI}.json'
                    #downlink_path = f'{EXP_FOLDER}{exp}/downlink_{devEUI}.json'
        
               
                    with open(data_path,'w') as data_file:
                        exp_data.append(data["result"])
                        json.dump(exp_data, data_file)
                    '''
                    with open(downlink_path) as downlink_file:
                        if "downlinkFrame" in data["result"].keys():
                            print('It is a Downlink')
                            
                            #downlinks.append(data["result"])
                            json.dump(downlinks, downlink_file)
                    '''        
                    
                    #print(data)
                    
    else:
        print('Authorization Required')       

In [4]:
startDeviceFrameHandler('dead2483dead0002','exp_device_17')


KeyboardInterrupt: 